# Create Transects and Polygons from Centerline

Note: The user input centerline shape file must have a Coordinate Reference System (CRS) that is projected as opposed to geographic. A CRS is also known as a Spatial Reference System (SRS). Examples of a projected CRS are a state plane or Universal Transverse Mercator (UTM).


## Initialize

In [1]:
!jupyter nbextension enable --py --sys-prefix ipyleaflet
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import ipywidgets as widgets
from IPython.display import display, display_html
from ipyfilechooser import FileChooser

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: ok


## User Input Widgets

In [2]:
# Define User Input Widgets
lbl1 = widgets.HTML(
    value= '<b>Number of CE-QUAL-W2 Segments)</b>',
)

seg_no_input = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1000000,
    step=1,
    description='Segments:',
    disabled=False
)

lbl2 = widgets.HTML(
    value= '<b>Transect Width (units are the same as the CRS associated with the centerline shape file)</b>',
)

width_input = widgets.FloatText(
    value=0,
    description='Width:',
    disabled=False
)

lbl3 = widgets.HTML(
    value= '<b>Please enter local or mapped drive letter below where centerline shape file is located:</b>',
)

drive_letter = widgets.Text(
    value='C',
    placeholder='enter drive letter here',
    description='Drive Letter:',
    disabled=False
)

# Display Widgets
display(lbl1)
display(seg_no_input)
display(lbl2)
display(width_input)
display(lbl3)
display(drive_letter)

HTML(value='<b>Number of CE-QUAL-W2 Segments)</b>')

BoundedIntText(value=1, description='Segments:', max=1000000, min=1)

HTML(value='<b>Transect Width (units are the same as the CRS associated with the centerline shape file)</b>')

FloatText(value=0.0, description='Width:')

HTML(value='<b>Please enter local or mapped drive letter below where centerline shape file is located:</b>')

Text(value='C', description='Drive Letter:', placeholder='enter drive letter here')

## User Select File Widget

In [3]:
#Add colon after drive letter
drive_letter_str = drive_letter.value + r":"


#### Code from: "https://pypi.org/project/ipyfilechooser/" ####

# Create and display a FileChooser widget
fc = FileChooser(drive_letter_str)
fc.filter_pattern = '*.shp'
display(fc)
    
# Change defaults and reset the dialog
fc.default_path = drive_letter_str
fc.default_filename = 'example_CL.shp'
fc.reset()
    
# Shorthand reset
fc.reset(path=drive_letter_str, filename='example_CL.shp')
    
# Change hidden files
fc.show_hidden = True
    
# Show or hide folder icons
fc.use_dir_icons = True
    
# Switch to folder-only mode
fc.show_only_dirs = False
    
# Set a file filter pattern (uses https://docs.python.org/3/library/fnmatch.html)
fc.filter_pattern = '*.shp'
    
# Change the title (use '' to hide)
fc.title = '<b>Select Centerline Shape File</b>'

# Sample callback function
def change_title(chooser):
    chooser.title = '<b>Select Centerline Shape File</b>'

# Register callback function
fc.register_callback(change_title)

FileChooser(path='I:', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='False'…

In [4]:
#Number of CE-QUAL-W2 Segments (USER INPUT)
Seg_No = seg_no_input.value

#Cross Section Width
width = width_input.value  #(USER INPUT) [Units are the same as the projected CRS provided in the centerline shape file]

#Import shape file centerline (USER INPUT)
wrkDir = fc.selected_path
pth_cl = fc.selected
geo_df_cl = gpd.read_file(pth_cl)
geo_df_cl_cntrd = gpd.read_file(pth_cl)
geo_df_cl_bndry = gpd.read_file(pth_cl)
cl_crs = geo_df_cl.crs

## Find Vertices from Centerline

In [5]:
geo_df_cl['points'] = geo_df_cl.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
cl_list = geo_df_cl['points'].tolist()
cl_array = np.array(cl_list)
cl_array_x = cl_array[0,...,0]
cl_array_y = cl_array[0,...,1]

## Find total length of the input Centerline

In [6]:
n = 0
dist_total = 0

for row in cl_array_x:
    if n == 0:
        x_pre = cl_array_x[n]
        y_pre = cl_array_y[n]
        n = n + 1
    else:
        x_n = cl_array_x[n]
        y_n = cl_array_y[n]
        dist_seg = np.sqrt((x_n - x_pre)**2 + (y_n - y_pre)**2)
        dist_total = dist_total + dist_seg
        x_pre = x_n
        y_pre = y_n
        n = n + 1
        cl_total_vertices = n

## Find W2 transect mid-points and centerline segment bounding vertices along input centerline

In [7]:
Seg_Lngth = dist_total / Seg_No
n = 0
k = 0
Seg_total = 0
clpts_w2Seg_total = Seg_No + 1
clpts_w2Seg_shape = (clpts_w2Seg_total, 1)
clpts_w2Seg_Dist = np.zeros(clpts_w2Seg_shape)
clpts_w2Seg_X = np.zeros(clpts_w2Seg_shape)
clpts_w2Seg_Y = np.zeros(clpts_w2Seg_shape)
clpts_w2Seg_bnd_X1 = np.zeros(clpts_w2Seg_shape)
clpts_w2Seg_bnd_X2 = np.zeros(clpts_w2Seg_shape)
clpts_w2Seg_bnd_Y1 = np.zeros(clpts_w2Seg_shape)
clpts_w2Seg_bnd_Y2 = np.zeros(clpts_w2Seg_shape)

for row in clpts_w2Seg_Dist: 
    if k == 0:  #this if condition populates the data for the first point along the centerline
        x_pre = cl_array_x[n]
        y_pre = cl_array_y[n]
        x_n = cl_array_x[n + 1]
        y_n = cl_array_y[n + 1]
        np.put(clpts_w2Seg_Dist, [k], [0])
        np.put(clpts_w2Seg_X, [k], [x_pre])
        np.put(clpts_w2Seg_Y, [k], [y_pre])
        np.put(clpts_w2Seg_bnd_X1, [k], [x_pre])
        np.put(clpts_w2Seg_bnd_X2, [k], [x_n])
        np.put(clpts_w2Seg_bnd_Y1, [k], [y_pre])
        np.put(clpts_w2Seg_bnd_Y2, [k], [y_n])
        k = k + 1
    elif k == (clpts_w2Seg_total - 1):  #this if condition populates the data for the last point along the centerline
        x_pre = cl_array_x[cl_total_vertices - 2]
        y_pre = cl_array_y[cl_total_vertices - 2]
        x_n = cl_array_x[cl_total_vertices - 1]
        y_n = cl_array_y[cl_total_vertices - 1]        
        np.put(clpts_w2Seg_Dist, [k], [dist_total])
        np.put(clpts_w2Seg_X, [k], [x_n])
        np.put(clpts_w2Seg_Y, [k], [y_n])
        np.put(clpts_w2Seg_bnd_X1, [k], [x_pre])
        np.put(clpts_w2Seg_bnd_X2, [k], [x_n])
        np.put(clpts_w2Seg_bnd_Y1, [k], [y_pre])
        np.put(clpts_w2Seg_bnd_Y2, [k], [y_n])
    else:
        running_total = 0
        Seg_total = Seg_total + Seg_Lngth
        np.put(clpts_w2Seg_Dist, [k], [Seg_total])
        n = 0
        x_pre = cl_array_x[n]
        y_pre = cl_array_y[n]
        n = n + 1
        while  Seg_total >= running_total:  #this while statement determines if the centerline segment has a W2 transect mid-point along it
            x_n = cl_array_x[n]
            y_n = cl_array_y[n]
            dist_seg = np.sqrt((x_n - x_pre)**2 + (y_n - y_pre)**2)
            running_total = running_total + dist_seg
            np.put(clpts_w2Seg_bnd_X1, [k], [x_pre])
            np.put(clpts_w2Seg_bnd_X2, [k], [x_n])
            np.put(clpts_w2Seg_bnd_Y1, [k], [y_pre])
            np.put(clpts_w2Seg_bnd_Y2, [k], [y_n])    
            x_pre = x_n
            y_pre = y_n
            n = n + 1    
        running_total_pre = running_total - dist_seg
        D = Seg_total - running_total_pre
        x_pre = cl_array_x[n - 2]
        y_pre = cl_array_y[n - 2]
        dx_sign = np.sign(x_n - x_pre)
        if dx_sign == 0: #this if statement traps the undefined slope of the cl segment
            #dx = 0 #x-coor is the same as x_pre since the cl segment is vertical
            dy_sign = np.sign(y_n - y_pre)
            dy = dy_sign * D
            cl_ptX = x_pre
            cl_ptY = y_pre + dy
        else:
            m_seg = (y_n - y_pre)/(x_n - x_pre)
            b = y_pre - (m_seg * x_pre)
            dx = D/(np.sqrt(1 + m_seg**2))
            dx = dx_sign * dx
            cl_ptX = x_pre + dx
            cl_ptY = m_seg * cl_ptX + b
            
        np.put(clpts_w2Seg_X, [k], [cl_ptX])
        np.put(clpts_w2Seg_Y, [k], [cl_ptY])
        k = k + 1

## Find End Points in the Transect

In [8]:
n = 0
d = width/2

clpts_w2Seg_m_seg = np.zeros((clpts_w2Seg_Y.shape))
clpts_w2Seg_m = np.zeros((clpts_w2Seg_Y.shape))
clpts_w2Seg_ptX2_pos = np.zeros((clpts_w2Seg_Y.shape))
clpts_w2Seg_ptY2_pos = np.zeros((clpts_w2Seg_Y.shape))
clpts_w2Seg_ptX2_neg = np.zeros((clpts_w2Seg_Y.shape))
clpts_w2Seg_ptY2_neg = np.zeros((clpts_w2Seg_Y.shape))
width_Check_array = np.zeros((clpts_w2Seg_Y.shape))


for row in clpts_w2Seg_X:
    segX1 = clpts_w2Seg_bnd_X1[n]
    segX2 = clpts_w2Seg_bnd_X2[n]
    segY1 = clpts_w2Seg_bnd_Y1[n]
    segY2 = clpts_w2Seg_bnd_Y2[n]
    ptX1 = clpts_w2Seg_X[n] #This is the same value as row
    ptY1 = clpts_w2Seg_Y[n]
    dx_seg = segX2 - segX1
    dy_seg = segY2 - segY1
    if dx_seg == 0:
        ptX2_pos = ptX1 + d
        ptX2_neg = ptX1 - d
        ptY2_pos = ptY1
        ptY2_neg = ptY1
    elif dy_seg == 0:
        ptX2_pos = ptX1
        ptX2_neg = ptX1
        ptY2_pos = ptY1 + d
        ptY2_neg = ptY1 - d
    else:
        m_seg = (dy_seg)/(dx_seg)
        m = -1 / m_seg    #This is the slope perpendicular to the CenterLine segment
        b = ptY1 - (m * ptX1)
        dX = d/(np.sqrt(1 + m**2))
        ptX2_pos = ptX1 + dX
        ptX2_neg = ptX1 - dX
        ptY2_pos = m * ptX2_pos + b
        ptY2_neg = m * ptX2_neg + b
    width_Check = np.sqrt((ptX2_pos - ptX2_neg)**2 + (ptY2_pos - ptY2_neg)**2)
    if ((segX2 - segX1) * m) > (segY2 - segY1):
        ptX2_pos_crss = ptX2_pos
        ptY2_pos_crss = ptY2_pos
        ptX2_neg_crss = ptX2_neg
        ptY2_neg_crss = ptY2_neg    
    else:
        ptX2_neg_crss = ptX2_pos
        ptY2_neg_crss = ptY2_pos
        ptX2_pos_crss = ptX2_neg
        ptY2_pos_crss = ptY2_neg
    np.put(clpts_w2Seg_m_seg, [n], [m_seg])
    np.put(clpts_w2Seg_m, [n], [m])
    np.put(clpts_w2Seg_ptX2_pos, [n], [ptX2_pos_crss])
    np.put(clpts_w2Seg_ptY2_pos, [n], [ptY2_pos_crss])
    np.put(clpts_w2Seg_ptX2_neg, [n], [ptX2_neg_crss])
    np.put(clpts_w2Seg_ptY2_neg, [n], [ptY2_neg_crss])
    np.put(width_Check_array, [n], [width_Check])
    n = n + 1

## Create Geopanda Data Frame for Transect Lines

In [9]:
#Create transect end point array
transect_pts_A = np.column_stack((clpts_w2Seg_Dist, clpts_w2Seg_ptX2_pos, clpts_w2Seg_ptY2_pos))
transect_pts_B = np.column_stack((clpts_w2Seg_Dist, clpts_w2Seg_ptX2_neg, clpts_w2Seg_ptY2_neg))
transect_pts = np.concatenate((transect_pts_A, transect_pts_B), axis=0)

#Create pandas data frame for transect end points
df_transects = pd.DataFrame(data=transect_pts, columns=["cl_Distance", "Transect_Xcoor", "Transect_Ycoor"])

#Zip the coordinates into a point object and convert to a geopanda data frame
geometry = [Point(xy) for xy in zip(df_transects.Transect_Xcoor, df_transects.Transect_Ycoor)]
geo_df_transect_pts = gpd.GeoDataFrame(df_transects, geometry=geometry)

#Create geopanda data frame for transect lines
geo_df_transect_lines = geo_df_transect_pts.groupby(['cl_Distance'])['geometry'].apply(lambda x: LineString(x.tolist()))
geo_df_transect_lines = gpd.GeoDataFrame(geo_df_transect_lines, geometry='geometry', crs = cl_crs)


In [10]:
# https://www.geeksforgeeks.org/check-if-two-given-line-segments-intersect/

# A Python3 program to find if 2 given line segments intersect or not 
  
class Point: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y 
  
# Given three colinear points p, q, r, the function checks if  
# point q lies on line segment 'pr'  
def onSegment(p, q, r): 
    if ( (q.x <= max(p.x, r.x)) and (q.x >= min(p.x, r.x)) and 
           (q.y <= max(p.y, r.y)) and (q.y >= min(p.y, r.y))): 
        return True
    return False
  
def orientation(p, q, r): 
    # to find the orientation of an ordered triplet (p,q,r) 
    # function returns the following values: 
    # 0 : Colinear points 
    # 1 : Clockwise points 
    # 2 : Counterclockwise 
      
    # See https://www.geeksforgeeks.org/orientation-3-ordered-points/amp/  
    # for details of below formula.  
      
    val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y)) 
    if (val > 0): 
          
        # Clockwise orientation 
        return 1
    elif (val < 0): 
          
        # Counterclockwise orientation 
        return 2
    else: 
          
        # Colinear orientation 
        return 0
  
# The main function that returns true if  
# the line segment 'p1q1' and 'p2q2' intersect. 
def doIntersect(p1,q1,p2,q2): 
      
    # Find the 4 orientations required for  
    # the general and special cases 
    o1 = orientation(p1, q1, p2) 
    o2 = orientation(p1, q1, q2) 
    o3 = orientation(p2, q2, p1) 
    o4 = orientation(p2, q2, q1) 
  
    # General case 
    if ((o1 != o2) and (o3 != o4)): 
        return True
  
    # Special Cases 
  
    # p1 , q1 and p2 are colinear and p2 lies on segment p1q1 
    if ((o1 == 0) and onSegment(p1, p2, q1)): 
        return True
  
    # p1 , q1 and q2 are colinear and q2 lies on segment p1q1 
    if ((o2 == 0) and onSegment(p1, q2, q1)): 
        return True
  
    # p2 , q2 and p1 are colinear and p1 lies on segment p2q2 
    if ((o3 == 0) and onSegment(p2, p1, q2)): 
        return True
  
    # p2 , q2 and q1 are colinear and q1 lies on segment p2q2 
    if ((o4 == 0) and onSegment(p2, q1, q2)): 
        return True
  
    # If none of the cases 
    return False


## Create and populate CE-QUAL-W2 polygons point arrays

In [11]:
chck_plygn_arry = np.array([0,1,2,3])
n = 1 #starting "for loop" at second record of the "clpts_w2Seg_pts#2_***" arrays to facilitate calling previous transect points
clpts_w2Segplygn_shape = (Seg_No, 1)
w2Seg_plygn_pt1_X = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_pt2_X = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_pt3_X = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_pt4_X = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_pt1_Y = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_pt2_Y = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_pt3_Y = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_pt4_Y = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_cldist = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_ID = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_convex = np.zeros((clpts_w2Segplygn_shape))
w2Seg_plygn_intersect = np.zeros((clpts_w2Segplygn_shape))
for row in w2Seg_plygn_pt1_X:
    pt1_X = clpts_w2Seg_ptX2_neg[n]
    pt1_Y = clpts_w2Seg_ptY2_neg[n]
    pt2_X = clpts_w2Seg_ptX2_pos[n]
    pt2_Y = clpts_w2Seg_ptY2_pos[n]
    pt3_X = clpts_w2Seg_ptX2_pos[n - 1]
    pt3_Y = clpts_w2Seg_ptY2_pos[n - 1]
    pt4_X = clpts_w2Seg_ptX2_neg[n - 1]
    pt4_Y = clpts_w2Seg_ptY2_neg[n - 1]
    cldist = clpts_w2Seg_Dist[n]
    plygn_ID = n
    #The following "for" statement checks for Concave CE-QUAL-W2 polygons
    for row in chck_plygn_arry:
        plygn_chck = \
        np.sign((pt2_X - pt1_X) * (pt3_Y - pt2_Y) - (pt3_X - pt2_X) * (pt2_Y - pt1_Y)) + \
        np.sign((pt3_X - pt2_X) * (pt4_Y - pt3_Y) - (pt4_X - pt3_X) * (pt3_Y - pt2_Y)) + \
        np.sign((pt4_X - pt3_X) * (pt1_Y - pt4_Y) - (pt1_X - pt4_X) * (pt4_Y - pt3_Y)) + \
        np.sign((pt1_X - pt4_X) * (pt2_Y - pt1_Y) - (pt2_X - pt1_X) * (pt1_Y - pt4_Y)) 
        if plygn_chck == 4:
            convex_plygn = 1
        else:
            convex_plygn = 0
    
    
    #The following "for" statement checks if polygon line segment pt2 to pt3 crosses any centerline segment
    k = 0
    intersect_cnt_pt2pt3 = 0
    for row in cl_array_x:
        if k == 0:
            k = k + 1
        else:
            p1 = Point(pt2_X, pt2_Y) 
            q1 = Point(pt3_X, pt3_Y) 
            p2 = Point(cl_array_x[k], cl_array_y[k]) 
            q2 = Point(cl_array_x[k - 1], cl_array_y[k - 1]) 

            if doIntersect(p1, q1, p2, q2): 
                intersect_cnt_pt2pt3 =  intersect_cnt_pt2pt3 + 1
            k = k + 1
 
    #The following "for" statement checks if polygon line segment pt4 to pt1 crosses any centerline segment
    k = 0
    intersect_cnt_pt4pt1 = 0
    for row in cl_array_x:
        if k == 0:
            k = k + 1
        else:
            p1 = Point(pt4_X, pt4_Y) 
            q1 = Point(pt1_X, pt1_Y) 
            p2 = Point(cl_array_x[k], cl_array_y[k]) 
            q2 = Point(cl_array_x[k - 1], cl_array_y[k - 1]) 

            if doIntersect(p1, q1, p2, q2): 
                intersect_cnt_pt4pt1 =  intersect_cnt_pt4pt1 + 1
            k = k + 1
        
    intersect_cnt = intersect_cnt_pt2pt3 + intersect_cnt_pt4pt1
    if intersect_cnt > 0:
        cl_intersect = 1
    else:
        cl_intersect = 0
    
    np.put(w2Seg_plygn_pt1_X, [n - 1], [pt1_X])
    np.put(w2Seg_plygn_pt2_X, [n - 1], [pt2_X])
    np.put(w2Seg_plygn_pt3_X, [n - 1], [pt3_X])
    np.put(w2Seg_plygn_pt4_X, [n - 1], [pt4_X])
    np.put(w2Seg_plygn_pt1_Y, [n - 1], [pt1_Y])
    np.put(w2Seg_plygn_pt2_Y, [n - 1], [pt2_Y])
    np.put(w2Seg_plygn_pt3_Y, [n - 1], [pt3_Y])
    np.put(w2Seg_plygn_pt4_Y, [n - 1], [pt4_Y])
    np.put(w2Seg_plygn_cldist, [n - 1], [cldist])
    np.put(w2Seg_plygn_ID, [n - 1], [plygn_ID])
    np.put(w2Seg_plygn_convex, [n - 1], [convex_plygn])
    np.put(w2Seg_plygn_intersect, [n - 1], [cl_intersect])
    n = n + 1

## Create Geopanda Data Frame for W2 Polygon Segments

In [12]:
#Assemble polygon points for use in pandas
polygon_pts_A = np.column_stack((w2Seg_plygn_ID, w2Seg_plygn_cldist, w2Seg_plygn_pt1_X, w2Seg_plygn_pt1_Y, w2Seg_plygn_convex))
polygon_pts_B = np.column_stack((w2Seg_plygn_ID, w2Seg_plygn_cldist, w2Seg_plygn_pt2_X, w2Seg_plygn_pt2_Y, w2Seg_plygn_convex))
polygon_pts_C = np.column_stack((w2Seg_plygn_ID, w2Seg_plygn_cldist, w2Seg_plygn_pt3_X, w2Seg_plygn_pt3_Y, w2Seg_plygn_convex))
polygon_pts_D = np.column_stack((w2Seg_plygn_ID, w2Seg_plygn_cldist, w2Seg_plygn_pt4_X, w2Seg_plygn_pt4_Y, w2Seg_plygn_convex))
polygon_pts = np.concatenate((polygon_pts_A, polygon_pts_B, polygon_pts_C, polygon_pts_D), axis=0)

#Create pandas data frame for W2 polygon segments
df_polygon = pd.DataFrame(data=polygon_pts, columns=["w2SegID", "cl_Distance", "plygn_Xcoor", "plygn_Ycoor", "convex"])
df_polygon.to_excel("df_polygoncheck_coordinates.xlsx")

#Zip the coordinates into a point object and convert to a geopanda data frame
geometry = [Point(xy) for xy in zip(df_polygon.plygn_Xcoor, df_polygon.plygn_Ycoor)]
geo_df_polygon_pts = gpd.GeoDataFrame(df_polygon, geometry=geometry)

#Create geopanda data frame for W2 polygon segments
geo_df_polygon = geo_df_polygon_pts.groupby(['w2SegID'])['geometry'].apply(lambda x: Polygon(x.tolist()))
geo_df_polygon = gpd.GeoDataFrame(geo_df_polygon, geometry='geometry', crs = cl_crs)

#Add attributes to W2 polygon segments
geo_df_polygon['w2SegID'] = w2Seg_plygn_ID
geo_df_polygon['cl_Distance'] = w2Seg_plygn_cldist
geo_df_polygon['convex'] = w2Seg_plygn_convex
geo_df_polygon['cl_intersect'] = w2Seg_plygn_intersect

#Create geopanda data frame for W2 polygon segment concave check
geo_df_polygon_check = geo_df_polygon[geo_df_polygon['convex']==0]

#Create geopanda data frame for W2 polygon segment that the centerline intersects
geo_df_polygon_intersect = geo_df_polygon[geo_df_polygon['cl_intersect']==1]

TypeError: __init__() missing 1 required positional argument: 'y'

# Map Centerline, W2 Segment Transects, & W2 Segment Polygons)

## Initialize for Mapping

In [ ]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
 LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
 MarkerCluster, Heatmap, SearchControl,  FullScreenControl, ScaleControl, 
 MeasureControl)
from ipywidgets import Text, HTML
from branca.colormap import linear
import json

## Create Map Layers, Add Map Controls, and Display Map

In [ ]:
#Provide initial center point
geo_df_cl_cntrd['cl_centroid'] = geo_df_cl_cntrd.centroid
geo_df_cl_cntrd = geo_df_cl_cntrd.set_geometry('cl_centroid')
geo_df_cl_cntrd_mapCRS = geo_df_cl_cntrd.to_crs(4326)
geo_df_cl_cntrd_mapCRS['points'] = geo_df_cl_cntrd_mapCRS.apply(lambda x: [y for y in x['cl_centroid'].coords], axis=1)
cntrd_list = geo_df_cl_cntrd_mapCRS['points'].tolist()
cntrd_array = np.array(cntrd_list)
cntrd_array_y = cntrd_array[0,...,0]
cntrd_array_x = cntrd_array[0,...,1]
center = [cntrd_array_x[0], cntrd_array_y[0]]

#Provide initial zoom level and find bounds of centerline for "fit_bounds" map method
zoom = 6
geo_df_cl_bndry['bndry'] = geo_df_cl_bndry.to_crs(4326).boundary
bndry_list = geo_df_cl_bndry['bndry'].tolist()
bndry_array = np.array(bndry_list)
bndry_array_y = bndry_array[0,...,0]
bndry_array_x = bndry_array[0,...,1]
map_bounds = ((bndry_array_x[1], bndry_array_y[1]), (bndry_array_x[0], bndry_array_y[0]))

#Create map and provide basemap
m = Map(basemap=basemaps.Esri.WorldImagery, center=center, zoom=zoom)
m.fit_bounds(map_bounds)

#Create and add spatial data layers to the map
map_cl = GeoData(geo_dataframe = geo_df_cl.to_crs(4326),
                   style={'color': 'yellow', 'opacity':3, 'weight':1.9, 'dashArray':'0', 'fillOpacity':1},
                   hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
                   name = 'Centerline')

map_transects = GeoData(geo_dataframe = geo_df_transect_lines.to_crs(4326),
                   style={'color': 'red', 'opacity':3, 'weight':1.9, 'dashArray':'0', 'fillOpacity':1},
                   hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
                   name = 'W2 Transects')

map_polygons = GeoData(geo_dataframe = geo_df_polygon.to_crs(4326),
                   style={'color': 'blue', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'green' , 'fillOpacity': 0.2},
                   name = 'W2 Segments')

map_polygons_concave = GeoData(geo_dataframe = geo_df_polygon_check.to_crs(4326),
                   style={'color': 'orange', 'fillColor': '#ffa500', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'green' , 'fillOpacity': 0.2},
                   name = 'W2 Segments Concave Check')

map_polygons_intersect = GeoData(geo_dataframe = geo_df_polygon_intersect.to_crs(4326),
                   style={'color': 'purple', 'fillColor': '#800080', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'green' , 'fillOpacity': 0.2},
                   name = 'W2 Segments CL Intersect Check')

#geo_df_polygon.to_file("temp_polygons.geojson", driver='GeoJSON')

#w2Seg_plygn_ID_f = w2Seg_plygn_ID.flatten('F')
#w2Seg_plygn_convex_f = w2Seg_plygn_convex.flatten('F')
#convex_dict = {}
#for A, B in zip(w2Seg_plygn_ID_f, w2Seg_plygn_convex_f):
#    convex_dict[A] = B

#map_polygon_check = Choropleth(geo_data = geo_df_polygon.to_crs(4326),
#                        choro_data=convex_dict,
#                        key_on='w2SegID',
#                        colormap=linear.YlOrRd_04,
#                        border_color='black',
#                        style={'fillOpacity': 0.6, 'dashArray': '5, 5'})

#m.add_layer(map_polygon_check)
m.add_layer(map_polygons_intersect)
m.add_layer(map_polygons_concave)
m.add_layer(map_polygons)
m.add_layer(map_transects)
m.add_layer(map_cl)


#Add map controls to the map
m.add_control(LayersControl())
m.add_control(FullScreenControl())
m.add_control(ScaleControl(position='bottomleft'))

measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'meters'
)
m.add_control(measure)

measure.completed_color = 'red'

measure.add_length_unit('feet', 3.28084, 4)
measure.secondary_length_unit = 'feet'

measure.add_area_unit('sqfeet', 10.7639, 4)
measure.secondary_area_unit = 'sqfeet'


#Display the map
print("NOTE: map can be zoomed to a rectangular area specified by dragging the mouse while pressing the shift key")
m
